In [1]:
!pip install transformers
!pip install torch
!pip install pandas

In [2]:
pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 11.0.0
    Uninstalling pillow-11.0.0:
      Successfully uninstalled pillow-11.0.0
  Attempting uninstall: typer
    Found existing installation: typer 0.15.1
    Uninstalling typer-0.15.1:
      Successfully uninstalled typer-0.15.1


In [2]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [13]:
data = pd.read_csv('/content/generated_hypotheses.csv')

In [14]:
# function to evaluate hypotheses
def evaluate_hypothesis(summary, hypothesis, category):
    # prompt for evaluation
    prompt = (
        f"Based on the following summary: '{summary}', evaluate the following hypothesis: "
        f"'{hypothesis}' on the category '{category}' on a scale from 0 to 10. "
        "Provide only the score, with no introduction or additional explanation."
    )

    # request to the model
    stream = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
        messages=[{"role": "user", "content": prompt}],
        stream=True,
    )

    score = ""
    for chunk in stream:
        score += (chunk.choices[0].delta.content or "")

    return float(score.strip())

# list of categories to evaluate
categories = ["novelty", "excitement", "feasibility", "effectiveness"]

# evaluate human hypotheses
for category in categories:
    column_name = f"human_{category}"
    data[column_name] = data.apply(
        lambda row: evaluate_hypothesis(row['cleaned_summary'], row['human_annotated_hypothesis'], category), axis=1
    )

# evaluate AI-generated hypotheses
for category in categories:
    column_name = f"ai_{category}"
    data[column_name] = data.apply(
        lambda row: evaluate_hypothesis(row['cleaned_summary'], row['together_ai_generated_hypothesis'], category), axis=1
    )

data.to_csv('/content/evaluated_hypotheses.csv', index=False)

print("Evaluations successfully completed and saved in 'evaluated_hypotheses.csv'")

Evaluations successfully completed and saved in 'evaluated_hypotheses.csv'
